In [2]:
from qdrant_client import QdrantClient
from langchain.vectorstores import Qdrant
from get_sentence_models import embedding_model_node_retrieval
# Kết nối với Qdrant
qdrant_url = ''
qdrant_api_key = ''
client = QdrantClient(qdrant_url,
api_key=qdrant_api_key
)  # Cập nhật địa chỉ nếu cần

# Hàm khởi tạo collection
def init_qdrant_collection(collection_name="node_spoke"):
    client.recreate_collection(
        collection_name=collection_name,
        vectors_config={"size": 384, "distance": "Cosine"},  # Điều chỉnh nếu dùng embedding khác
    )

# Dữ liệu nodes
nodes = [
    "Disease", "Food", "Gene", "Reaction", "Symptom", 
    "SideEffect", "Compound", "PharmacologicClass"
]

# Hàm lưu các node vào Qdrant
def save_nodes_to_qdrant(collection_name="node_spoke"):
    qdrant = Qdrant(client, collection_name, embedding_model_node_retrieval)

    # Tạo embeddings và lưu vào Qdrant
    for node in nodes:
        qdrant.add_texts([node])

    print("Lưu nodes vào Qdrant thành công!")

# Khởi tạo collection và lưu dữ liệu
init_qdrant_collection()
save_nodes_to_qdrant()


/tmp/ipykernel_1432510/4216086451.py:11: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


Lưu nodes vào Qdrant thành công!


In [4]:
relationships = [
    # Disease
    {"relationship": "ISA", "source": "Disease", "target": "Disease"},
    {"relationship": "RESEMBLES", "source": "Disease", "target": "Disease"},
    {"relationship": "ASSOCIATES", "source": "Disease", "target": "Gene"},
    {"relationship": "PRESENTS", "source": "Disease", "target": "Symptom"},
    
    # Food
    {"relationship": "CONTAINS", "source": "Food", "target": "Compound"},
    
    # Gene
    {"relationship": "MARKER", "source": "Gene", "target": "Disease"},
    
    # Reaction
    {"relationship": "CONSUMES", "source": "Reaction", "target": "Compound"},
    {"relationship": "PRODUCES", "source": "Reaction", "target": "Compound"},
    
    # Compound
    {"relationship": "TREATS", "source": "Compound", "target": "Disease"},
    {"relationship": "CONTRAINDICATES", "source": "Compound", "target": "Disease"},
    {"relationship": "INTERACTS", "source": "Compound", "target": "Food"},
    {"relationship": "PARTICIPATES", "source": "Compound", "target": "Reaction"},
    {"relationship": "CAUSES", "source": "Compound", "target": "Side Effect"},
    {"relationship": "ISA", "source": "Compound", "target": "Compound"},
    {"relationship": "PARTOF", "source": "Compound", "target": "Compound"},
    {"relationship": "HASROLE", "source": "Compound", "target": "Compound"},
    
    # Pharmacologic Class
    {"relationship": "INCLUDES", "source": "Pharmacologic Class", "target": "Compound"}
]

# Hàm lưu các relationship cùng metadata vào Qdrant
def save_relationships_to_qdrant(collection_name="relationship"):
    qdrant = Qdrant(client, collection_name, embedding_model_node_retrieval)

    texts = [rel["relationship"] for rel in relationships]  # Chỉ lấy tên relationship
    metadatas = [{"source": rel["source"], "target": rel["target"]} for rel in relationships]

    # Lưu vào Qdrant với metadata
    qdrant.add_texts(texts, metadatas=metadatas)
    print("Lưu relationships vào Qdrant thành công!")

# Đảm bảo collection tồn tại và lưu dữ liệu
init_qdrant_collection('relationship SPOKE')
save_relationships_to_qdrant('relationship SPOKE')

/tmp/ipykernel_1432510/4216086451.py:11: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


Lưu relationships vào Qdrant thành công!
